In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
### import csv's
hs_file = "Resources/highschool.csv"
hs = pd.read_csv(hs_file, encoding = 'cp437')
pov_file = "Resources/poverty.csv"
pov = pd.read_csv(pov_file, encoding = 'cp437')
police_file = "Resources/killings.csv"
police = pd.read_csv(police_file, encoding = 'cp437')

In [4]:
hs.head()

,Geographic Area,City,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6


In [5]:
#Clean Police File
clean_police = police[['date', 'armed', 'age','gender','race','city',
                            'state','signs_of_mental_illness','body_camera']].copy()
clean_police.head()

,date,armed,age,gender,race,city,state,signs_of_mental_illness,body_camera
0,02/01/15,gun,53.0,M,A,Shelton,WA,True,False
1,02/01/15,gun,47.0,M,W,Aloha,OR,False,False
2,03/01/15,unarmed,23.0,M,H,Wichita,KS,False,False
3,04/01/15,toy weapon,32.0,M,W,San Francisco,CA,True,False
4,04/01/15,nail gun,39.0,M,H,Evans,CO,False,False


In [17]:
clean_police['date'] = pd.to_datetime(clean_police.date)

In [32]:
pov_matched = pov.rename(columns={"Geographic Area": "state", "City": "city"}, errors="raise")

In [33]:
hs_matched = hs.rename(columns={"Geographic Area": "state", "City": "city"}, errors="raise")


In [34]:
hs_matched.head()

,state,city,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6


In [35]:
pov_matched.replace(to_replace='-', value=0, inplace=True)

In [36]:
# Create Postresql engine
rds_connection_string = "postgres:karen135@localhost:5432/PoliceKillings"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [37]:
engine.table_names()

['poverty_rate', 'hs_rate']

In [38]:
pov_matched.to_sql(name='poverty_rate', con=engine, if_exists='append', index=False)